### Naver 블로그 

블로그 텍스트 데이터 웹 크롤링 using selenium

In [ ]:
# 환경 setup

from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import re

In [ ]:
# url 추출하기 method정의

def get_urls(query, totalCount, startDate, endDate):
    url_list_recent = [] 
    
    if totalCount%7:
        lastPage = int(totalCount/7) + 1
    else:
        lastPage = int(totalCount/7)

    for i in range(1,lastPage+1):
        #url='https://section.blog.naver.com/Search/Post.naver?pageNo='+ str(i) + '&rangeType=PERIOD&orderBy=sim&startDate=2020-08-15&endDate=2021-08-15&keyword='+query
        url='https://section.blog.naver.com/Search/Post.naver?pageNo='+ str(i) + '&rangeType=PERIOD&orderBy=recentdate&startDate=2021-08-01&endDate=2021-08-15&keyword='+query
        driver.get(url)
        time.sleep(0.5)

        for j in range(1, 10): # 각 블로그 주소 저장
            try: 
                titles = driver.find_element_by_xpath('/html/body/ui-view/div/main/div/div/section/div[2]/div['+str(j)+']/div/div[1]/div[1]/a[1]')
                href = titles.get_attribute('href')
                url_list_recent.append(href)
            except:
                pass
            
    url_list_recent = list(set(url_list_recent))
    return url_list_recent

In [ ]:
# get_url 실행 및 csv로 저장하기

# 크롬 웹브라우저 실행
path = 'C:/SJL/GyeonggidoData_Course/Teamproject/Chrome/chromedriver.exe'
driver = webdriver.Chrome(path)

query = '"제로 웨이스트"'
startDate='2021-07-15'
endDate='2021-08-15'
totalCount=2500

url_list = get_urls(query, totalCount, startDate, endDate)
driver.close()

x = pd.Series(url_list)
x.to_csv('./zerowaste/naverblog_urls.csv', index=False)

len(url_list)

In [ ]:
date_list=[] 
like_list=[]
content_list = []
hashtag_list=[]

In [ ]:
# posting 내용 추출하기 method정의

def get_postings(u_list):
    url_list = list(u_list['URL']) 
    count=0

    for link in url_list: # 수집한 url 만큼 반복
        driver.get(link) # 해당 url로 이동
        try:
            driver.switch_to.frame('mainFrame')
            time.sleep(1.5)

            # date
            publish_date='.blog2_container'
            dates = driver.find_elements_by_css_selector(publish_date)
            datetime=""
            for d in dates: datetime+=d.text
            date_list.append(datetime)

            # like
            u_like='.wrap_postcomment'
            likes = driver.find_elements_by_css_selector(u_like)
            like_counts=""
            for k in likes[1:2]: like_counts+=k.text
            like_list.append(like_counts)

            # content
            overlays = ".se-component.se-text.se-l-default"
            contents = driver.find_elements_by_css_selector(overlays)
            posting_contents=""
            for content in contents:
                posting_contents+=content.text
            content_list.append(posting_contents)

            # hashtag
            try:
                htags='.wrap_tag'
                tags = driver.find_elements_by_css_selector(htags)
                hash_tags=""
                for h in tags: hash_tags+=h.text
            except:
                pass
            hashtag_list.append(hash_tags)

            count+=1
            if count%100==0:
                print(count, 'postings scraped')
        except:
            pass
    print("url counts: ",len(url_list))
    print("date counts: ",len(date_list))
    print("like counts: ", len(like_list))
    print("content counts: ",len(content_list))
    print("hashtag: ", len(hashtag_list))

    all_data = pd.DataFrame(data={'URL':url_list, 'Date':date_list, 'Like': like_list, 
            'Content':content_list, 'Hashtag':hashtag_list})    
        
    return all_data

In [ ]:
# 추출 dataframe내 불필요 문자/기호 전처리하는 method 정의

def preprocess_data(dfcopy):
    
    for idx in dfcopy.index:
        dfcopy.loc[idx,'Date'] = re.sub('[a-zA-Zㄱ-힗]','',str(dfcopy.loc[idx,'Date']))
        try:
            dfcopy.loc[idx,'Date'] = str(dfcopy.loc[idx,'Date']).split("・")[1]
        except:
            dfcopy.loc[idx,'Date'] = ""

        dfcopy.loc[idx,'Like'] = str(dfcopy.loc[idx,'Like']).split("\n")[0]
        dfcopy.loc[idx,'Like'] = re.findall("\d+",str(dfcopy.loc[idx,'Like']))
        try:
            dfcopy.loc[idx,'Like'] = dfcopy.loc[idx,'Like'][0]
        except:
            dfcopy.loc[idx,'Like'] = ""

        dfcopy.loc[idx,'Content'] = str(dfcopy.loc[idx,'Content']).replace('\n',' ')
        dfcopy.loc[idx,'Hashtag'] = str(dfcopy.loc[idx,'Hashtag']).replace('\n',' ')
        
    return dfcopy

In [ ]:
# url list 불러오기 및 posting 내용 가져오기 실행

u_list = pd.read_csv('./zerowaste/naverblog_urls.csv')

# 크롬 웹브라우저 실행
path = 'C:/SJL/GyeonggidoData_Course/Teamproject/Chrome/chromedriver.exe'
driver = webdriver.Chrome(path)

data = get_postings(u_list)
driver.close()

data_treated = preprocess_data(data)
data_treated.head()

data_treated.to_csv('./zerowaste/naverblogs_crawling_selenium2.csv',index=False, encoding="utf-8-sig")

In [ ]:
# final dataframe 구성 확인

data_treated.info()